In [25]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests as rq
from IPython.display import clear_output
import random
import time
print('Imported all packages')

Imported all packages


In [32]:

language_data = []
lang_links = []

def scrape(num_of_pages):
    for i in range(0, num_of_pages + 1):
        clear_output(wait=True)
        sleep_time = random.uniform(0.1, 0.5)
        
        print(f'Scraping page {i} out of {num_of_pages}')
        page_url = f'https://en.wal.unesco.org/discover/languages?text=&sort_by=title&page={i}'
        
        try:
            page = rq.get(page_url)
            page.raise_for_status() 
            
            page_soup = soup(page.text)
            links = page_soup.find_all('a', href=True)
            
            for link in links:
                href = link['href']
                
                if 'https://en.wal.unesco.org/languages/' in href:
                    lang_links.append(href)
            
            time.sleep(sleep_time)
        except rq.exceptions.RequestException as e:
            print(f"Error fetching page {i}: {e}")
            continue


In [ ]:
scrape(277)

Scraping page 55 out of 277


In [22]:
def extract_language_data(lang_links):
    #iterate through lang_links and gather language information using extract_language_info
    for link in lang_links:
        clear_output(wait=True)
        print(f'Language URL {link}')
        lang = rq.get(lang_url)
        if lang.status_code == 200:
            lang_soup = soup(lang.text)
            lang_name = lang_soup.select('h2')[3].get_text()
            lang_speakers = lang_soup.select('span')[6].get_text()
            glotto_div = lang_soup.find('div', class_='field__label', text='Glottocode')
            if glotto_div:
                glotto_item = glotto_div.find_next_sibling('div', class_='field__item')
                glottocode = glotto_item.get_text(strip=True)
            else:
                glottocode = 'none found'
            return {'name': lang_name, 'speakers': lang_speakers, 'glottocode': glottocode}
        else:
            print(f'Error at: {lang_url}')
            return None
        lang_info = extract_language_info(link)
        language_data.append(lang_info)
    
    clear_output()
    df = pd.DataFrame.from_dict(language_data)
    df.to_csv('unesco_wal.csv')
